In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/reviews.csv')
df.head()

,rating,sitter_image,end_date,text,owner_image,dogs,sitter,owner,start_date,sitter_phone_number,sitter_email,owner_phone_number,owner_email
0,5,http://placekitten.com/g/500/500?user=338,2013-04-08,Donec lacus justo luctus tellus nisl penatibus...,http://placekitten.com/g/500/500?user=12,Pinot Grigio,Lauren B.,Shelli K.,2013-02-26,12546478758,user4739@gmail.com,15817557107,user2555@verizon.net
1,5,http://placekitten.com/g/500/500?user=24,2012-05-17,Etiam morbi duis sed. Etiam netus torquent don...,http://placekitten.com/g/500/500?user=155,Rasty-CAT|Shogun|Annie,Leilani R.,Nancy L.,2012-04-03,13813114382,user7508@t-mobile.com,11826582052,user3444@t-mobile.com
2,2,http://placekitten.com/g/500/500?user=149,2012-07-28,Justo massa. Nulla purus. Justo lacus dolor lu...,http://placekitten.com/g/500/500?user=175,Mali,Sharon H.,Nelli B.,2012-06-22,17805637575,user7582@yahoo.com,14970255634,user7852@hotmail.com
3,2,http://placekitten.com/g/500/500?user=172,2013-05-31,Purus ipsum etiam condimentum volutpat sodales...,http://placekitten.com/g/500/500?user=168,Lego,Tasha J.,Mel P.,2013-05-05,18473545812,user7137@yahoo.com,15191086391,user4860@t-mobile.com
4,2,http://placekitten.com/g/500/500?user=230,2012-10-17,Ipsum nulla duis dapibus. Curae vitae maecenas...,http://placekitten.com/g/500/500?user=316,Rocco|Elvis|Teddy,Huan H.,Jim S.,2012-09-17,11362389237,user1233@yahoo.com,19715860916,user3007@verizon.net


In [3]:
len(df['owner_email'].unique())

189

In [4]:
pd.concat([
    df[df['owner'] == 'Tony L.'],
    df[df['sitter'] == 'Tony L.']
])

,rating,sitter_image,end_date,text,owner_image,dogs,sitter,owner,start_date,sitter_phone_number,sitter_email,owner_phone_number,owner_email
12,3,http://placekitten.com/g/500/500?user=153,2012-06-26,Ipsum etiam nam semper faucibus. Class lacus. ...,http://placekitten.com/g/500/500?user=158,layla|Boots|Hiwalani,Lyndsey B.,Tony L.,2012-06-10,13774343299,user1211@yahoo.com,11045881284,user6943@gmail.com
243,3,http://placekitten.com/g/500/500?user=88,2012-03-31,Lacus porta leo pede eleifend auctor sapien al...,http://placekitten.com/g/500/500?user=158,layla|Boots|Hiwalani,Pam L.,Tony L.,2012-03-16,12332041180,user9227@t-mobile.com,11045881284,user6943@gmail.com
221,4,http://placekitten.com/g/500/500?user=120,2012-03-26,Metus risus. Dolor risus vel elit ultrices mi ...,http://placekitten.com/g/500/500?user=156,Kashmere,Tony L.,Christina F.,2012-03-15,13786670381,user6074@t-mobile.com,17111106664,user5733@t-mobile.com
327,4,http://placekitten.com/g/500/500?user=120,2013-04-21,Neque vitae eros vulputate tellus libero velit...,http://placekitten.com/g/500/500?user=262,Ceasar,Tony L.,Sn M.,2013-02-07,13786670381,user6074@t-mobile.com,19020171691,user7531@gmail.com
349,5,http://placekitten.com/g/500/500?user=120,2012-05-28,Velit magna aliquam neque et felis. Justo cura...,http://placekitten.com/g/500/500?user=34,Roxy,Tony L.,Carrie M.,2012-03-28,13786670381,user6074@t-mobile.com,13608238869,user1050@hotmail.com
425,1,http://placekitten.com/g/500/500?user=120,2012-08-23,Vitae ipsum scelerisque mus nisi felis. Donec ...,http://placekitten.com/g/500/500?user=109,Schatzie|tabitha,Tony L.,Fahreta G.,2012-06-01,13786670381,user6074@t-mobile.com,12916239875,user8400@gmail.com
437,3,http://placekitten.com/g/500/500?user=120,2012-03-27,Dolor risus lacinia proin lacus magnis vitae d...,http://placekitten.com/g/500/500?user=222,Juno|Sandpiper|A.T.the All Terrain dog,Tony L.,Lindsey C.,2012-02-03,13786670381,user6074@t-mobile.com,14192864067,user7463@yahoo.com


In [5]:
len(df['sitter'].unique())

100

In [6]:
len(df['sitter_email'].unique())

100

In [7]:
len(df['owner_email'].unique())

189

In [8]:
df['sitter_email'].unique()

array(['user4739@gmail.com', 'user7508@t-mobile.com',
       'user7582@yahoo.com', 'user7137@yahoo.com', 'user1233@yahoo.com',
       'user9866@gmail.com', 'user7112@verizon.net', 'user1949@gmail.com',
       'user7736@t-mobile.com', 'user5363@verizon.net',
       'user7177@verizon.net', 'user2983@verizon.net',
       'user1211@yahoo.com', 'user2847@verizon.net',
       'user3697@verizon.net', 'user5135@t-mobile.com',
       'user1943@verizon.net', 'user1777@yahoo.com', 'user6713@gmail.com',
       'user9973@hotmail.com', 'user6604@yahoo.com', 'user8249@yahoo.com',
       'user8738@gmail.com', 'user7851@hotmail.com',
       'user9612@t-mobile.com', 'user4669@t-mobile.com',
       'user9082@t-mobile.com', 'user2807@hotmail.com',
       'user4671@yahoo.com', 'user7784@hotmail.com',
       'user1969@hotmail.com', 'user1945@yahoo.com', 'user3695@yahoo.com',
       'user5411@hotmail.com', 'user3251@yahoo.com',
       'user4917@t-mobile.com', 'user1075@gmail.com',
       'user8635@verizon.ne

In [9]:
len(set(df['sitter_email'].values + df['owner_email'].values))

498

In [10]:
sitter_email_set = set(df['sitter_email'].values)
owner_email_set = set(df['owner_email'].values)

print(len(sitter_email_set))
print(len(owner_email_set))

sitter_email_set.update(owner_email_set)
print(len(sitter_email_set))

100
189
289


In [11]:
print(df['owner_phone_number'].min())
print(df['owner_phone_number'].max())

print(df['sitter_phone_number'].min())
print(df['sitter_phone_number'].max())

11033269916
19907428841
11183717179
19955877682


In [12]:
df_ratings = pd.DataFrame(data={
    'ratings_score': df.groupby(['sitter'])['rating'].mean(),
    'stays': df.groupby(['sitter'])['sitter'].count(),
    'sitter': df['sitter'].unique(),
}, index=df['sitter'].unique())

df_ratings.head()

,ratings_score,sitter,stays
Lauren B.,2.571429,Lauren B.,7
Leilani R.,3.500000,Leilani R.,12
Sharon H.,2.750000,Sharon H.,4
Tasha J.,1.666667,Tasha J.,3
Huan H.,3.428571,Huan H.,7


In [13]:
df_scores = pd.DataFrame(data={
    'sitter': df['sitter'].unique(),
}, index=df['sitter'].unique())


df_scores['sitter_score'] = df_scores['sitter'].apply(
    lambda x: (sum(1 for let in {s for s in x.lower()} if let.isalpha()) / 26)*5)

df_scores.head()

,sitter,sitter_score
Lauren B.,Lauren B.,1.346154
Leilani R.,Leilani R.,1.153846
Sharon H.,Sharon H.,1.153846
Tasha J.,Tasha J.,0.961538
Huan H.,Huan H.,0.769231


In [14]:
try:
    df_scores = df_scores.drop(columns=['sitter'])
except ValueError:
    pass

try:
    df_ratings = df_ratings.drop(columns=['sitter'])
except ValueError:
    pass

df_overall = pd.concat([df_ratings, df_scores], axis=1)
df_overall.head()

,ratings_score,stays,sitter_score
Lauren B.,2.571429,7,1.346154
Leilani R.,3.500000,12,1.153846
Sharon H.,2.750000,4,1.153846
Tasha J.,1.666667,3,0.961538
Huan H.,3.428571,7,0.769231


In [15]:
def overall_score(row):
    if row['stays'] == 0:
        return row['sitter_score']
    elif row['stays'] >= 10:
        return row['ratings_score']
    return row['stays'] * .1 * row['ratings_score'] + (row['sitter_score'] * (1 - (row['stays'] * .1)))

df_overall['overall'] = df_overall.apply(overall_score, axis=1)

df_overall.head()

,ratings_score,stays,sitter_score,overall
Lauren B.,2.571429,7,1.346154,2.203846
Leilani R.,3.500000,12,1.153846,3.500000
Sharon H.,2.750000,4,1.153846,1.792308
Tasha J.,1.666667,3,0.961538,1.173077
Huan H.,3.428571,7,0.769231,2.630769


In [16]:
df_raw = pd.DataFrame({
    'stays': [i for i in range(13)],
    'ratings_score': [5 for i in range(13)],
    'sitter_score': [2.5 for i in range(13)]
})
df_raw['overall'] = df_raw.apply(overall_score, axis=1)
# working
# df_raw

In [17]:
df_joined_with_scores = df.join(df_overall, on='sitter')

In [18]:
df_joined_with_scores.to_csv('../data/reviews-with-scores.csv')

### Check for overlap between sitters and owners

`Name` is the unique identifier

In [19]:
df_sitter = pd.DataFrame({
    'sitter': df['sitter'].unique(),
    'sitter_count': df.groupby(['sitter'])['sitter'].count()
}, index=df['sitter'].unique())

df_sitter.head()

,sitter,sitter_count
Lauren B.,Lauren B.,7
Leilani R.,Leilani R.,12
Sharon H.,Sharon H.,4
Tasha J.,Tasha J.,3
Huan H.,Huan H.,7


In [20]:
df_owner = pd.DataFrame({
    'owner': df['owner'].unique(),
    'owner_count': df.groupby(['owner'])['owner'].count()
}, index=df['owner'].unique()) 

df_owner.head()

,owner,owner_count
Shelli K.,Shelli K.,2
Nancy L.,Nancy L.,1
Nelli B.,Nelli B.,2
Mel P.,Mel P.,2
Jim S.,Jim S.,5


In [21]:
df_combined = df_sitter.join(df_owner)

df_combined[df_combined.owner.notnull()]

,sitter,sitter_count,owner,owner_count
Tony L.,Tony L.,5,Tony L.,2.0
